# Problem Statement:

## Predicting speed of the Permanent Magnet Synchronous Motor(PMSM) given other sensor measurements during operation..

# Import necessary libraries

In [ ]:
#conda install conda=23.5.0

In [ ]:
#pip install tensorflow

In [ ]:
#pip install mlxtend

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split,GridSearchCV,KFold,cross_val_score
from sklearn import linear_model
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet,RANSACRegressor
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from keras.models import Sequential
from keras.layers import Dense

plt.rcParams['figure.figsize']=(16,12)
plt.rcParams['figure.dpi']=250
import warnings
warnings.filterwarnings('ignore')

# Load Dataset

In [ ]:
em=pd.read_csv("temperature_data.csv")
em.head(10)

In [ ]:
em.tail(10)

# EDA

In [ ]:
em.shape

In [ ]:
em.info()

In [ ]:
# computes various summary statistics, excluding NaN values

em.describe().T

In [ ]:
#Check null values

em.isnull().sum()

In [ ]:
#Print the duplicated values.

em[em.duplicated()]

#### Observation: 
There are no duplicate values and null values present in this dataset.

# Correlation

In [ ]:
# Pairlot for visualizing the correlation between features

sns.pairplot(em)

#### Since there is no  proper pattern visible in the plot correlation matrix is used to get the magnitude of the correlations between the features

In [ ]:
# for computing correlations
em.corr()

In [ ]:
corr = em.corr()
#Plot figsize
fig, ax = plt.subplots(figsize=(12, 8))
#Generate Heat Map, allow annotations and place floats in map
sns.heatmap(corr, cmap='magma', annot=True, fmt=".2f")
#Apply xticks
plt.xticks(range(len(corr.columns)), corr.columns);
#Apply yticks
plt.yticks(range(len(corr.columns)), corr.columns)
#show plot
plt.show()

### Observations:

There is no significant relation between the dependent and the idpendent variables except u_q and i_d

u_q is positively correlated with the motorspeed while i_d is affecting negatively

## Univariate plots

### To check the distribution of data

In [ ]:
em..iloc[:,:-1].hist(figsize = (35,25))
plt.show()

In [ ]:
#sns.distplot(np.sqrt(em['motor_speed']))

In [ ]:
sns.set(style="whitegrid", font_scale=1.8)
plt.subplots(figsize = (25,8))
sns.countplot('profile_id',data=em).set_title('count of profile_id')
plt.show()

## Outlier Detection

In [ ]:
#from seaborn.axisgrid import share_axis
em.iloc[:,:-1].plot(kind='box',subplots=True,layout=(4,4))
plt.show()

### Observation:

"Ambient" , "torque" and "i_q" features has the outliers on both the ends

"pm" and "u_d" has outliers on the top end



### Transforming the features pm and u_d

In [ ]:
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
sns.boxplot(np.sqrt(em['u_d']))
plt.title("u_d")

plt.subplot(1,2,2)
sns.boxplot(np.sqrt(em['pm']))
plt.title('pm')
plt.show()

In [ ]:
# Transforming the features u_d amd pm
em['u_d']=np.sqrt(np.abs(em['u_d']))
em['pm']=np.sqrt(np.abs(em['pm']))

Hence the outliers in the tge featurs pm and u_d can be treated by applying square root them

#### Outliers in the feature "ambient"

In [ ]:
em['ambient'].describe()

In [ ]:
#Quartile 1
q1_am=em['ambient'].quantile(0.25)
print('Quatile 1:',q1_am)

#median
med_am=em['ambient'].median()
print("Median",med_am)

#Quartile 3
q3_am=em['ambient'].quantile(0.75)
print('Quatile 3:',q3_am)

#Interquartile Range
print("Inter quartile Range: ",q3_am-q1_am)

#Upper limit
upp_am=q3_am+1.5*(q3_am-q1_am)
print('Upper Limit: ',upp_am)

#Lower limit
low_am=q1_am-1.5*(q3_am-q1_am)
print('Lower Limit: ',low_am)

In [ ]:
# outliers in ambient
em[(em['ambient'] > upp_am) | (em['ambient'] < low_am)]

#### Outliers the feature "torque"

In [ ]:
em['torque'].describe()

In [ ]:
#Quartile 1
q1_tq=em['torque'].quantile(0.25)
print('Quatile 1:',q1_tq)

#median
med_tq=em['torque'].median()
print("Median",med_tq)

#Quartile 3
q3_tq=em['torque'].quantile(0.75)
print('Quatile 3:',q3_tq)

#Interquartile Range
print("Inter quartile Range: ",q3_tq-q1_tq)

#Upper limit
upp_tq=q3_tq+1.5*(q3_tq-q1_tq)
print('Upper Limit: ',upp_tq)

#Lower limit
low_tq=q1_tq-1.5*(q3_tq-q1_tq)
print('Lower Limit: ',low_tq)

In [ ]:
# outliers in torque
em[(em['torque'] > upp_tq) | (em['torque'] < low_tq)]

#### Outliers in the feature "i_q"

In [ ]:
em['i_q'].describe()

In [ ]:
#Quartile 1
q1_iq=em['i_q'].quantile(0.25)
print('Quatile 1:',q1_iq)

#median
med_iq=em['i_q'].median()
print("Median",med_iq)

#Quartile 3
q3_iq=em['i_q'].quantile(0.75)
print('Quatile 3:',q3_iq)

#Interquartile Range
print("Inter quartile Range: ",q3_iq-q1_iq)

#Upper limit
upp_iq=q3_iq+1.5*(q3_iq-q1_iq)
print('Upper Limit: ',upp_iq)

#Lower limit
low_iq=q1_iq-1.5*(q3_iq-q1_iq)
print('Lower Limit: ',low_iq)

In [ ]:
# outliers in i_q
em[(em['i_q'] > upp_iq) | (em['i_q'] < low_iq)]

Rows with all the 3 columns as outliers

In [ ]:
em[((em['ambient'] > upp_am) | (em['ambient'] < low_am)) & ((em['torque'] > upp_tq) | (em['torque'] < low_tq)) & ((em['i_q'] > upp_iq) | (em['i_q'] < low_iq))]

In [ ]:
# value counts of profile_id that contains outliers in all the 3 columns
em[((em['ambient'] > upp_am) | (em['ambient'] < low_am)) & ((em['torque'] > upp_tq) | (em['torque'] < low_tq)) & ((em['i_q'] > upp_iq) | (em['i_q'] < low_iq))]['profile_id'].value_counts()

Rows with atleast 1 out of 3 columns containg outlier

In [ ]:
#Rows with outliers
em[((em['ambient'] > upp_am) | (em['ambient'] < low_am)) | ((em['torque'] > upp_tq) | (em['torque'] < low_tq)) | ((em['i_q'] > upp_iq) | (em['i_q'] < low_iq))]


There are 1,80,510 outliers totally present in the dataset after transforming u_d and pm columns.So we are not deleting those rowes but replacing the outliers greater than upper limit with the upper-limit and with lower limit for those data that are lesser than lower limit

#### Treating Outlier

In [ ]:
# Replacing outliers greater than upper-limit with the upper limit
em.loc[em['ambient']>=upp_am,'ambient']=med_am

# Replacing outliers lower than lower-limit with the lower limit
em.loc[em['ambient']<=low_am,'ambient']=med_am

In [ ]:
# Plotting the feature ambient
plt.figure(figsize=(6,4))
em.boxplot('ambient')
plt.show()

In [ ]:
# Replacing outliers greater than upper-limit with the upper limit
em.loc[em['torque']>=upp_tq,'torque']=med_tq

# Replacing outliers lower than lower-limit with the lower limit
em.loc[em['torque']<=low_tq,'torque']=med_tq

In [ ]:
# Plotting the feature torque
plt.figure(figsize=(6,4))
em.boxplot('torque')
plt.show()

In [ ]:
# Replacing outliers greater than upper-limit with the upper limit
em.loc[em['i_q']>=upp_iq,'i_q']=med_iq

# Replacing outliers lower than lower-limit with the lower limit
em.loc[em['i_q']<=low_iq,'i_q']=med_iq

In [ ]:
# Plotting the feature i_q
plt.figure(figsize=(6,4))
em.boxplot('i_q')
plt.show()

In [ ]:
em.plot(kind='box',subplots=True,layout=(3,5))
plt.show()

There are no outliers present in the dataset after treating the outliers

In [ ]:
corr = em.corr()
#Plot figsize
fig, ax = plt.subplots(figsize=(12, 8))
#Generate Heat Map, allow annotations and place floats in map
sns.heatmap(corr, cmap='magma', annot=True, fmt=".2f")
#Apply xticks
plt.xticks(range(len(corr.columns)), corr.columns);
#Apply yticks
plt.yticks(range(len(corr.columns)), corr.columns)
#show plot
plt.show()

#### Observation:
There is no much change in the correlation values after treating the outliers.

The independent features have high correlation values leading to multi-collinearity and only 2 features 'u_q' and 'i_d' are having significant correlations with the dependent feature 'motor_speed' 

##### To solve this problem we shall check for VIF.

### Variance Inflation Factors

In [ ]:
# dropping the column 'profile_id' as it doesnot contributes in predicting the motor_speed
em1=em.drop('profile_id',axis=1)

em1.head()

In [ ]:
#Split the data into independent variable X and dependent Y

X = em1.drop('motor_speed',axis=1)
y = em1['motor_speed']

In [ ]:
X

In [ ]:
y

In [ ]:
# VIF Calculation

rsq_am = smf.ols('ambient ~ coolant+u_d+u_q+torque+i_d+i_q+pm+stator_yoke+stator_tooth+stator_winding',data=em1).fit().rsquared  
vif_am = 1/(1-rsq_am)

rsq_cnt = smf.ols('coolant ~ ambient+u_d+u_q+torque+i_d+i_q+pm+stator_yoke+stator_tooth+stator_winding',data=em1).fit().rsquared  
vif_cnt = 1/(1-rsq_cnt)

rsq_ud = smf.ols('u_d ~ ambient+coolant+u_q+torque+i_d+i_q+pm+stator_yoke+stator_tooth+stator_winding',data=em1).fit().rsquared  
vif_ud = 1/(1-rsq_ud)

rsq_uq = smf.ols('u_q ~ ambient+coolant+u_d+torque+i_d+i_q+pm+stator_yoke+stator_tooth+stator_winding',data=em1).fit().rsquared  
vif_uq = 1/(1-rsq_uq)

rsq_tq = smf.ols('torque ~ ambient+coolant+u_d+u_q+i_d+i_q+pm+stator_yoke+stator_tooth+stator_winding',data=em1).fit().rsquared  
vif_tq = 1/(1-rsq_tq)

rsq_id = smf.ols('i_d ~ ambient+coolant+u_d+u_q+torque+i_q+pm+stator_yoke+stator_tooth+stator_winding',data=em1).fit().rsquared  
vif_id = 1/(1-rsq_id)

rsq_iq = smf.ols('i_q ~ ambient+coolant+u_d+u_q+torque+i_d+pm+stator_yoke+stator_tooth+stator_winding',data=em1).fit().rsquared  
vif_iq = 1/(1-rsq_iq)

rsq_pm = smf.ols('pm ~ ambient+coolant+u_d+u_q+torque+i_d+i_q+stator_yoke+stator_tooth+stator_winding',data=em1).fit().rsquared  
vif_pm = 1/(1-rsq_pm)

rsq_sy = smf.ols('stator_yoke ~ ambient+u_d+u_q+torque+i_d+i_q+pm+stator_tooth+stator_winding',data=em1).fit().rsquared  
vif_sy = 1/(1-rsq_sy)

rsq_st = smf.ols('stator_tooth ~ ambient+coolant+u_d+u_q+torque+i_d+i_q+pm+stator_yoke+stator_winding',data=em1).fit().rsquared  
vif_st = 1/(1-rsq_st)

rsq_sw = smf.ols('stator_winding ~ ambient+coolant+u_d+u_q+torque+i_d+i_q+pm+stator_yoke+stator_tooth',data=em1).fit().rsquared  
vif_sw = 1/(1-rsq_sw)

In [ ]:
#Storing vif values in a data frame
d1 = {'Variables':X.columns,'VIF':[vif_am,vif_cnt,vif_ud,vif_uq,vif_tq,vif_id,vif_iq,vif_pm,vif_sy,vif_st,vif_sw]}
Vif_frame = pd.DataFrame(d1)  
Vif_frame

The VIF of coolant,torque,i_q,stator_yoke,stator_tooth,stator_winding is too high so shall remove these features for further analysis.

In [ ]:
X.drop(['coolant','stator_yoke','stator_tooth','stator_winding'],axis=1,inplace=True)
X

##### We shall further use Feature Selection as there was no much correlation between the motor_speed to other independent variables

## Feature Selection

In [ ]:
#Lasso

l1 = Lasso(alpha=0.3)
l1.fit(X,y)

l1_coeff = pd.DataFrame() 
l1_coeff["Features"] = X.columns 
l1_coeff['Coefficient Estimate'] = pd.Series(l1.coef_) 

print(l1_coeff) 

In [ ]:
#Elastic net

en = ElasticNet()
en.fit(X,y)

en_coeff = pd.DataFrame() 
en_coeff["Columns"] = X.columns 
en_coeff['Coefficient Estimate'] = pd.Series(en.coef_) 
en_coeff

##### We shall continue with the features u_q and i_d for model-building

In [ ]:
# Scaling the features u_q and i_d
sc_X = StandardScaler()
X_scale= sc_X.fit_transform(X[['u_q','i_d']])
X_scale

In [ ]:
# Splitting the train test data
x_train, x_test, y_train, y_test = train_test_split(X_scale,y,test_size = 0.2,random_state=10)

## Linear Regression

In [ ]:
lr=LinearRegression()
lr.fit(x_train,y_train)
ypred1=lr.predict(x_test)

r2_ln=r2_score(y_test,ypred1)
ln_mse=mean_squared_error(y_test,ypred1)
ln_rmse=np.sqrt(ln_mse)

print("R_square : ",r2_ln)
print("Mean Square Error: ",ln_mse)
print("Root Mean Square Error: ",ln_rmse)

## Robust Regression

In [ ]:
rb=RANSACRegressor(base_estimator=LinearRegression())
rb.fit(x_train, y_train)
ypred_rb=rb.predict(x_test)

r2_rb=r2_score(y_test,ypred_rb)
rb_mse=mean_squared_error(y_test,ypred_rb)
rb_rmse=np.sqrt(rb_mse)

print("R_square : ",r2_rb)
print("Mean Square Error: ",rb_mse)
print("Root Mean Square Error: ",rb_rmse)

## KNN

In [ ]:
knn=KNeighborsRegressor()
knn.fit(x_train, y_train)
ypred_knn=knn.predict(x_test)

r2_knn=r2_score(y_test,ypred_knn)
knn_mse=mean_squared_error(y_test,ypred_knn)
knn_rmse=np.sqrt(knn_mse)

print("R_square : ",r2_knn)
print("Mean Square Error: ",knn_mse)
print("Root Mean Square Error: ",knn_rmse)

## Decision Tree

In [ ]:
dt = DecisionTreeRegressor()
dt.fit(x_train,y_train)
ypred_dt=dt.predict(x_test)
r2_dt=r2_score(y_test,ypred_dt)
dt_mse=mean_squared_error(y_test,ypred_dt)
dt_rmse=np.sqrt(dt_mse)
print("R_square : ",r2_dt)
print("Mean Square Error: ",dt_mse)
print("Root Mean Square Error: ",dt_rmse)

## Random Forest

In [ ]:
model = RandomForestRegressor()
model.fit(x_train,y_train)
ypred_rf = model.predict(x_test)


In [ ]:
r2_rf=r2_score(y_test,ypred_rf)
rf_mse=mean_squared_error(y_test,ypred_rf)
rf_rmse=np.sqrt(rf_mse)
print("R_square : ",r2_rf)
print("Mean Square Error: ",rf_mse)
print("Root Mean Square Error: ",rf_rmse)

## Polynomial Regression

In [ ]:
poly_reg = PolynomialFeatures(degree = 2)
poly_reg.fit_transform(x_train)
x_train_poly=poly_reg.transform(x_train)
x_test_poly=poly_reg.transform(x_test)

plr = LinearRegression()
plr.fit(x_train_poly,y_train)
plr.predict(x_test_poly)
ypred_plr=plr.predict(x_test_poly)

r2_plr=r2_score(y_test,ypred_plr)
plr_mse=mean_squared_error(y_test,ypred_plr)
plr_rmse=np.sqrt(plr_mse)

print("R_square : ",r2_plr)
print("Mean Square Error: ",plr_mse)
print("Root Mean Square Error: ",plr_rmse)

## Support vector Machine

In [ ]:
svr = SVR(kernel='rbf')

In [ ]:
svr.fit(x_train, y_train)

In [ ]:
ypred_svm = svr.predict(x_test)

In [ ]:
r2_svm=r2_score(y_test,ypred_svm)
svm_mse=mean_squared_error(y_test,ypred_svm)
svm_rmse=np.sqrt(svm_mse)

print("R_square : ",r2_svm)
print("Mean Square Error: ",svm_mse)
print("Root Mean Square Error: ",svm_rmse)

##  Neural Networks

In [ ]:
model=Sequential()
model.add(Dense(10,input_dim=2,activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='Adam')

In [ ]:
# Fit the model
model.fit(x_train,y_train)

In [ ]:
ypred_nn=model.predict(x_test)

In [ ]:
r2_nn=r2_score(y_test,ypred_nn)
nn_mse=mean_squared_error(y_test,ypred_nn)
nn_rmse=np.sqrt(nn_mse)

print("R_square : ",r2_nn)
print("Mean Square Error: ",nn_mse)
print("Root Mean Square Error: ",nn_rmse)

## Storing the results in the table format

In [ ]:
result_df=pd.DataFrame()
result_df['Models']=['Linear Regression','Robust Regression','KNN','Decision Tree','Random Forest','Polynomial Regression','Neural Network']
result_df['R_square values']=[r2_ln,r2_rb,r2_knn,r2_dt,r2_rf,r2_plr,r2_nn]
result_df['Mean Square Error']=[ln_mse,rb_mse,knn_mse,dt_mse,rf_mse,plr_mse,nn_mse]
result_df['Root Mean Square Error']=[ln_rmse,rb_rmse,knn_rmse,dt_rmse,rf_rmse,plr_rmse,nn_rmse]
result_df

In [ ]:
plt.figure(figsize=(4,4))
plt.style.use('Solarize_Light2')
plt.subplot(2,1,1)
plt.bar(result_df['Models'],result_df['R_square values'])
plt.title('Models with R_square values',fontdict={'color':'blue','size':10})
plt.xticks(rotation=90)
print()
plt.figure(figsize=(4,4))
plt.subplot(2,1,2)
plt.bar(result_df['Models'],result_df['Root Mean Square Error'])
plt.title('Models with Root Mean Square Error',fontdict={'color':'blue','size':10})
plt.xticks(rotation=90)
plt.show()

##### Here KNN regression model is giving the higher R-square and lower Mean-square value  hence we are choosing KNN fot the Model Building

### Tunning the Hyperparameter

In [ ]:
params={'n_neighbors':[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]}
knn=KNeighborsRegressor()

model=GridSearchCV(knn,params,cv=5)
model.fit(x_train,y_train)
model.best_params_

In [ ]:
# Model building
model=KNeighborsRegressor(n_neighbors=19)
model.fit(x_train,y_train)
ypred=model.predict(x_test)

# Calculating R2 score,MSE and RMSE
r2=r2_score(y_test,ypred)
mse=mean_squared_error(y_test,ypred)
rmse=np.sqrt(mse)

# Printing R2 score,MSE and RMSE
print("R_square : ",np.round(r2,2))
print("Mean Square Error: ",np.round(mse,2))
print("Root Mean Square Error: ",np.round(rmse,2))

In [ ]:
# Storing y_test and ypred values in a Dataframe
df=pd.DataFrame()
df['Actual speed']=y_test
df['Predicted speed']=ypred
df.reset_index(drop=True)

In [ ]:
plt.figure(figsize=(6,4))
sns.distplot(df['Actual speed'],hist=False,color='green')
sns.distplot(df['Predicted speed'],hist=False,color='red')
plt.legend(['Actual speed','Predicted speed'])
plt.title("Actual speed v/s Predicted speed",fontdict={'color':'blue','size':20})
plt.show()

#### Here the actual and predicted values are alomost overlapping this shows that the model is performing well

In [ ]:
# Plotting Actual and Predicted values
plt.scatter(x=y_test,y=ypred)
plt.title("Actual speed v/s Predicted speed",fontdict={'color':'blue','size':20})
plt.xlabel("Actual speed",fontdict={'color':'blue','size':15})
plt.ylabel("Predicted speed",fontdict={'color':'blue','size':15})

## Validating the Model
#### Using KFold

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kfold = KFold(n_splits=10)
results = cross_val_score(model, X_scale,y, cv=kfold)
results

In [ ]:
results.mean()

In [ ]:
mod=[1,2,3,4,5,6,7,8,9,10]

In [ ]:
plt.figure(figsize=(4,4))
plt.style.use('ggplot')
plt.subplot(2,1,1)
plt.bar(mod,results)
plt.title('Model wise Results',fontdict={'color':'r','size':14})
plt.xlabel('MODELS',fontdict={'color':'r','size':8})
plt.ylabel('RESULTS',fontdict={'color':'r','size':8})
plt.show()
print()


### Final model 

In [ ]:
final_model=KNeighborsRegressor(n_neighbors=19)
final_model.fit(X_scale,y)
ypred=final_model.predict(X_scale)

In [ ]:
plt.figure(figsize=(6,4))
sns.distplot(y,hist=False,color='green')
sns.distplot(ypred,hist=False,color='red')
plt.legend(['Actual speed','Predicted speed'])
plt.title("Actual speed v/s Predicted speed",fontdict={'color':'blue','size':20})
plt.show()